In [459]:
# ---- LIBRARY IMPORTS ----
import pandas as pd
import datetime
import requests
from datetime import datetime
from haversine import haversine

In [460]:
# ---- Reading in datasets ----
usecols = [index for index in range(1, 14)]
dtype = {'ride_id': "string", 'rideable_type': "string", 'started_at': "string", 'ended_at': "string", 'start_station_name': "string",
         'start_station_id': "string", 'end_station_name': "string", 'end_station_id': "string", 'start_lat': float,
         'start_lng': float, 'end_lat': float, 'end_lng': float, 'member_casual': "string"}
df = pd.read_csv('data/202407-citibike-tripdata_5.csv', usecols=usecols, dtype=dtype)

In [461]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,557426B1CE207702,electric_bike,2024-07-17 19:30:15.471,2024-07-17 19:34:22.646,Sullivan St & Washington Sq,5721.01,E 12 St & 3 Ave,5788.12,40.730477,-73.999061,40.732233,-73.988900,member
1,A60E366FEE8B51E8,electric_bike,2024-07-15 19:17:52.523,2024-07-15 19:25:24.399,Sullivan St & Washington Sq,5721.01,Forsyth St & Grand St,5382.07,40.730477,-73.999061,40.717798,-73.993161,member
2,ACA30207F80B5DF9,classic_bike,2024-07-27 13:34:48.531,2024-07-27 13:49:55.913,W 37 St & 10 Ave,6611.02,Perry St & Bleecker St,5922.07,40.756604,-73.997901,40.735354,-74.004831,member
3,B3A138C956E9A729,electric_bike,2024-07-23 11:01:36.895,2024-07-23 11:06:02.796,Melrose St & Broadway,4801.04,Nostrand Ave & Myrtle Ave,4707.04,40.697481,-73.935877,40.695270,-73.952381,member
4,EA0BFE3B299B0672,electric_bike,2024-07-16 19:19:15.034,2024-07-16 19:50:39.140,Melrose St & Broadway,4801.04,W 27 St & 7 Ave,6257.06,40.697481,-73.935877,40.746650,-73.993920,member


# Data Cleansing
The following operations are required for data cleaning

1. Encode `rideable_type` and `member_casual`
2. Create fields for `start_date`, `start_time`, `end_date`, `end_time`
3. Create field for trip duration
4. Create distance between stations using the [Manhattan's Distance](https://www.datacamp.com/tutorial/manhattan-distance)
5. Drop any rows with missing station IDs

## Encoding Binary Variables
The `rideable_type` and `member_casual` fields will use dummy encoding using pandas' [`get_dummies`](https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html) method.

In [462]:
df = pd.get_dummies(data=df, columns=['rideable_type', 'member_casual'], dtype="int", drop_first=True)

## Drop any NA values
We want to drop any values with `Null` for missing station IDs, names, ride IDs, coordinates or names because if any of them are `Null`, then we cannot understand what what on during a ride.

In [463]:
df.dropna(subset=['ride_id', 'started_at', 'ended_at', 'start_station_name', 'start_station_id',
                  'end_station_id', 'end_station_name', 'start_lat', 'end_lat', 'end_lng', 'end_lng'], inplace=True)

## Create fields for `start_date`, `start_time`, `end_date`, `end_time`

In [464]:
df['started_at'] = pd.to_datetime(df['started_at'])
df['ended_at'] = pd.to_datetime(df['ended_at'])
df['start_date'] = pd.to_datetime(df['started_at'], format="%Y-%m-%d")
df['end_date'] = pd.to_datetime(df['ended_at'], format="%Y-%m-%d")
df['start_time'] =  pd.to_datetime(df['started_at'], format="%H:%M:%S.%f")
df['end_time'] = pd.to_datetime(df['ended_at'], format="%H:%M:%S.%f")

## Create field for trip_duration
We will now create the `trip_duration` field that describes the trip duration in seconds.

In [465]:
trip_duration = df['ended_at'] - df['started_at']
get_trip_seconds = lambda val: val.total_seconds()
df['trip_duration'] = trip_duration.map(get_trip_seconds)

## Get Distance Between All Stations
We now will capture the distances between all stations using the [Haversine Distance](https://pypi.org/project/haversine/), in meters.

In [466]:
def haversine_map(row):
    start_coordinates = (row['start_lat'], row['start_lng'])
    end_coordinates = (row['end_lat'], row['end_lng'])
    dist = haversine(start_coordinates, end_coordinates, unit="m")
    return dist

df['ride_dist'] = df.apply(haversine_map, axis=1)

In [458]:
df.columns

Index(['ride_id', 'started_at', 'ended_at', 'start_station_name',
       'start_station_id', 'end_station_name', 'end_station_id', 'start_lat',
       'start_lng', 'end_lat', 'end_lng', 'rideable_type_electric_bike',
       'member_casual_member', 'start_date', 'end_date', 'start_time',
       'end_time', 'trip_duration', 'ride_dist'],
      dtype='object')